### テスト

In [1]:
import sympy

sympy.init_printing()

P = sympy.Matrix([
    [1, 2]
])

print(P)


Matrix([[1, 2]])


### 逆行列の計算

In [4]:
# 変数を定義
x, y = sympy.symbols('x y')

# 係数行列を構築
coeff_matrix = sympy.Matrix([
    [2, 3, 6],
    [4, -2, 8]
])

# 連立方程式を解く
solution = sympy.linsolve(coeff_matrix, x, y)

print("Solution:", solution)

Solution: {(9/4, 1/2)}
